In [1]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print(len(text))

1115394


In [2]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [21]:
# Create encoding for text
all_chars = sorted(list(set(text)))
vocab_size = len(all_chars)
print(len(all_chars))
print(''.join(all_chars))

65

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [11]:
# Define encoding and decoding functions for this set

char_to_integer = { ch:i for i, ch in enumerate(all_chars) }
integer_to_char = { v:k for k, v in char_to_integer.items() }
encode = lambda s: [char_to_integer[c] for c in s]
decode = lambda l: ''.join([integer_to_char[n] for n in l])

print(encode("test word shibboleth"))
print(decode(encode("test word shibboleth")))

[58, 43, 57, 58, 1, 61, 53, 56, 42, 1, 57, 46, 47, 40, 40, 53, 50, 43, 58, 46]
test word shibboleth


In [12]:
# Use encoding function to encode entire dataset, store in pytorch tensor and do 90/10 training/validation split (first 90/last 10)

import torch
dataset_tensor = torch.tensor(encode(text), dtype=torch.long)
print(dataset_tensor.shape, dataset_tensor.dtype)
print(dataset_tensor[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [14]:
n = int(0.9*len(dataset_tensor))
train_data = dataset_tensor[:n]
val_data = dataset_tensor[n:]

In [16]:
# Block size determines the size of what the transformer sees
block_size = 8
print(train_data[:block_size+1])
# Print example for context and training label
example_data = train_data[:block_size+1]
assert len(example_data) == block_size + 1
for i in range(block_size):
    context = example_data[:i+1]
    target = example_data[i+1]
    print(f"When input is {context}, target is {target}")

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])
When input is tensor([18]), target is 47
When input is tensor([18, 47]), target is 56
When input is tensor([18, 47, 56]), target is 57
When input is tensor([18, 47, 56, 57]), target is 58
When input is tensor([18, 47, 56, 57, 58]), target is 1
When input is tensor([18, 47, 56, 57, 58,  1]), target is 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]), target is 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), target is 58


In [20]:
torch.manual_seed(1337)
batch_size = 4 # Independent sequences processed in parallel
block_size = 8 # 1 context window the transformer sees

def get_batch(split: str):
    data = train_data if split == 'train' else val_data
    indices = torch.randint(len(data) - block_size, (batch_size,))
    contexts = torch.stack([data[i:i+block_size] for i in indices])
    targets = torch.stack([data[i+1:i+1+block_size] for i in indices])
    return contexts, targets

sample_contexts, sample_targets = get_batch('train')
print('inputs')
print(sample_contexts.shape)
print(sample_contexts)
print('targets')
print(sample_targets.shape)
print(sample_targets)
print('=========')

for b in range(batch_size): # parallelism
    for t in range(block_size): # variable context length
        sample_context = sample_contexts[b, :t+1]
        sample_target = sample_targets[b, t]
        print(f"When input is {sample_context.tolist()}, target is {sample_target.tolist()}")


inputs
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
When input is [24], target is 43
When input is [24, 43], target is 58
When input is [24, 43, 58], target is 5
When input is [24, 43, 58, 5], target is 57
When input is [24, 43, 58, 5, 57], target is 1
When input is [24, 43, 58, 5, 57, 1], target is 46
When input is [24, 43, 58, 5, 57, 1, 46], target is 43
When input is [24, 43, 58, 5, 57, 1, 46, 43], target is 39
When input is [44], target is 53
When input is [44, 53], target is 56
When input is [44, 53, 56], target is 1
When input is [44, 53, 56, 1], target is 58
When input is [44, 53, 56, 1, 58], target is 46
When input is [44, 53, 56, 1, 58, 46], ta

In [22]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets):
        return self.token_embedding_table(idx)

m = BigramLanguageModel(vocab_size)
out = m(sample_contexts, sample_targets)
print(out.shape)

torch.Size([4, 8, 65])
